Colab para generar features (extraido de la ayudita que dio el profe Alejandro en Zullip)

In [ ]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

"A diferencia de la competencia 1, ahora vamos a usar una base de datos en disco. Todo va a ser más lento, pero se va a poder trabajar sin problemas en máquinas de menor capacidad. La DB va a queda en el archivo dmeyf.db, lo probé en un colab básico y funcionó sin problemas."


In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:////content/dmeyf.db

"Cargamos el dataset, y como tenemos la base en el disco, no debemos cargarlo más, la próxima se puede usar directamente"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('/content/drive/MyDrive/DM/DMEyF/competencia_02.csv.gz')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


#Features históricas

Ver todos los campos

In [ ]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'competencia_02';

,column_name,data_type
0,numero_de_cliente,BIGINT
1,foto_mes,BIGINT
2,active_quarter,BIGINT
3,cliente_vip,BIGINT
4,internet,BIGINT
...,...,...
150,Visa_mconsumototal,DOUBLE
151,Visa_cconsumos,BIGINT
152,Visa_cadelantosefectivo,BIGINT
153,Visa_mpagominimo,DOUBLE


Calcular el valor del mes anterior de un campo


In [ ]:
%%time
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_02
limit 10

CPU times: user 3.9 s, sys: 307 ms, total: 4.21 s
Wall time: 2.56 s


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter
0,29188883,201901,43,NaN
1,29188883,201902,45,43.0
2,29188883,201903,39,45.0
3,29188883,201904,38,39.0
4,29188883,201905,39,38.0
5,29188883,201906,41,39.0
6,29188883,201907,34,41.0
7,29188883,201908,30,34.0
8,29188883,201909,25,30.0
9,29188883,201910,30,25.0


Si embargo puede resultar incómodo escribir constantemente el over partition sobre todo si se buscan aplicar muchas veces para distintas funciones. Para reducir el código se puede usar la siguiente sintaxis




In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6
  , max(ctrx_quarter) over ventana_6 as ctrx_quarter_max_6
  , min(ctrx_quarter) over ventana_6 as ctrx_quarter_min_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 5 preceding and current row)
limit 10

,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_media_6,ctrx_quarter_max_6,ctrx_quarter_min_6
0,29188883,201901,43,43.000000,43,43
1,29188883,201902,45,44.000000,45,43
2,29188883,201903,39,42.333333,45,39
3,29188883,201904,38,41.250000,45,38
4,29188883,201905,39,40.800000,45,38
5,29188883,201906,41,40.833333,45,38
6,29188883,201907,34,39.333333,45,34
7,29188883,201908,30,36.833333,41,30
8,29188883,201909,25,34.500000,41,25
9,29188883,201910,30,33.166667,41,25


Un caso más, que ni me voy a molestar en explicar que significa...



In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 5 preceding and current row)
limit 10

,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_slope_6
0,29188883,201901,43,NaN
1,29188883,201902,45,2.000000
2,29188883,201903,39,-2.000000
3,29188883,201904,38,-2.100000
4,29188883,201905,39,-1.500000
5,29188883,201906,41,-0.828571
6,29188883,201907,34,-1.371429
7,29188883,201908,30,-1.571429
8,29188883,201909,25,-2.828571
9,29188883,201910,30,-2.771429


In [ ]:
campos = ['ctrx_quarter', 'mpasivos_margen', 'mautoservicio' , 'ctarjeta_debito_transacciones' , 'mtransferencias_emitidas' , 'ctransferencias_emitidas' , 'mpayroll' , 'cpayroll_trx', 'mcomisiones_mantenimiento', 'mcomisiones_otras', 'mcomisiones', 'mcaja_ahorro', 'mcuentas_saldo', 'mtarjeta_visa_consumo','mprestamos_personales']
print(campos)
print(len(campos))

['ctrx_quarter', 'mpasivos_margen', 'mautoservicio', 'ctarjeta_debito_transacciones', 'mtransferencias_emitidas', 'ctransferencias_emitidas', 'mpayroll', 'cpayroll_trx', 'mcomisiones_mantenimiento', 'mcomisiones_otras', 'mcomisiones', 'mcaja_ahorro', 'mcuentas_saldo', 'mtarjeta_visa_consumo', 'mprestamos_personales']
15


###6 meses

In [ ]:
### Query para 6 meses
len(campos)
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_6 as {campo}_slope_6"
  nuevos_features += f", lag({campo}, 1) over ventana_6 as {campo}_lag_1"
  nuevos_features += f", lag({campo}, 2) over ventana_6 as {campo}_lag_2"
  nuevos_features += f", lag({campo}, 3) over ventana_6 as {campo}_lag_3"
  nuevos_features += f", lag({campo}, 4) over ventana_6 as {campo}_lag_4"
  nuevos_features += f", lag({campo}, 5) over ventana_6 as {campo}_lag_5"
  nuevos_features += f", lag({campo}, 6) over ventana_6 as {campo}_lag_6"
  nuevos_features += f", avg({campo}) over ventana_6 as {campo}_media_6"

print(nuevos_features)


, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6, lag(ctrx_quarter, 1) over ventana_6 as ctrx_quarter_lag_1, lag(ctrx_quarter, 2) over ventana_6 as ctrx_quarter_lag_2, lag(ctrx_quarter, 3) over ventana_6 as ctrx_quarter_lag_3, lag(ctrx_quarter, 4) over ventana_6 as ctrx_quarter_lag_4, lag(ctrx_quarter, 5) over ventana_6 as ctrx_quarter_lag_5, lag(ctrx_quarter, 6) over ventana_6 as ctrx_quarter_lag_6, avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_6 as mpasivos_margen_slope_6, lag(mpasivos_margen, 1) over ventana_6 as mpasivos_margen_lag_1, lag(mpasivos_margen, 2) over ventana_6 as mpasivos_margen_lag_2, lag(mpasivos_margen, 3) over ventana_6 as mpasivos_margen_lag_3, lag(mpasivos_margen, 4) over ventana_6 as mpasivos_margen_lag_4, lag(mpasivos_margen, 5) over ventana_6 as mpasivos_margen_lag_5, lag(mpasivos_margen, 6) over ventana_6 as mpasivos_margen_lag_6, avg(mpasivos_marge

el resultado anterior lo uso para esta nueva query

In [ ]:
%%time
%%sql
create or replace table competencia_02 as
SELECT
  *
, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6, lag(ctrx_quarter, 1) over ventana_6 as ctrx_quarter_lag_1, lag(ctrx_quarter, 2) over ventana_6 as ctrx_quarter_lag_2, lag(ctrx_quarter, 3) over ventana_6 as ctrx_quarter_lag_3, lag(ctrx_quarter, 4) over ventana_6 as ctrx_quarter_lag_4, lag(ctrx_quarter, 5) over ventana_6 as ctrx_quarter_lag_5, lag(ctrx_quarter, 6) over ventana_6 as ctrx_quarter_lag_6, avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_6 as mpasivos_margen_slope_6, lag(mpasivos_margen, 1) over ventana_6 as mpasivos_margen_lag_1, lag(mpasivos_margen, 2) over ventana_6 as mpasivos_margen_lag_2, lag(mpasivos_margen, 3) over ventana_6 as mpasivos_margen_lag_3, lag(mpasivos_margen, 4) over ventana_6 as mpasivos_margen_lag_4, lag(mpasivos_margen, 5) over ventana_6 as mpasivos_margen_lag_5, lag(mpasivos_margen, 6) over ventana_6 as mpasivos_margen_lag_6, avg(mpasivos_margen) over ventana_6 as mpasivos_margen_media_6
, regr_slope(mautoservicio, cliente_antiguedad) over ventana_6 as mautoservicio_slope_6, lag(mautoservicio, 1) over ventana_6 as mautoservicio_lag_1, lag(mautoservicio, 2) over ventana_6 as mautoservicio_lag_2, lag(mautoservicio, 3) over ventana_6 as mautoservicio_lag_3, lag(mautoservicio, 4) over ventana_6 as mautoservicio_lag_4, lag(mautoservicio, 5) over ventana_6 as mautoservicio_lag_5, lag(mautoservicio, 6) over ventana_6 as mautoservicio_lag_6, avg(mautoservicio) over ventana_6 as mautoservicio_media_6
, regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_6 as ctarjeta_debito_transacciones_slope_6, lag(ctarjeta_debito_transacciones, 1) over ventana_6 as ctarjeta_debito_transacciones_lag_1, lag(ctarjeta_debito_transacciones, 2) over ventana_6 as ctarjeta_debito_transacciones_lag_2, lag(ctarjeta_debito_transacciones, 3) over ventana_6 as ctarjeta_debito_transacciones_lag_3, lag(ctarjeta_debito_transacciones, 4) over ventana_6 as ctarjeta_debito_transacciones_lag_4, lag(ctarjeta_debito_transacciones, 5) over ventana_6 as ctarjeta_debito_transacciones_lag_5, lag(ctarjeta_debito_transacciones, 6) over ventana_6 as ctarjeta_debito_transacciones_lag_6, avg(ctarjeta_debito_transacciones) over ventana_6 as ctarjeta_debito_transacciones_media_6
, regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_6 as mtransferencias_emitidas_slope_6, lag(mtransferencias_emitidas, 1) over ventana_6 as mtransferencias_emitidas_lag_1, lag(mtransferencias_emitidas, 2) over ventana_6 as mtransferencias_emitidas_lag_2, lag(mtransferencias_emitidas, 3) over ventana_6 as mtransferencias_emitidas_lag_3, lag(mtransferencias_emitidas, 4) over ventana_6 as mtransferencias_emitidas_lag_4, lag(mtransferencias_emitidas, 5) over ventana_6 as mtransferencias_emitidas_lag_5, lag(mtransferencias_emitidas, 6) over ventana_6 as mtransferencias_emitidas_lag_6, avg(mtransferencias_emitidas) over ventana_6 as mtransferencias_emitidas_media_6
, regr_slope(ctransferencias_emitidas, cliente_antiguedad) over ventana_6 as ctransferencias_emitidas_slope_6, lag(ctransferencias_emitidas, 1) over ventana_6 as ctransferencias_emitidas_lag_1, lag(ctransferencias_emitidas, 2) over ventana_6 as ctransferencias_emitidas_lag_2, lag(ctransferencias_emitidas, 3) over ventana_6 as ctransferencias_emitidas_lag_3, lag(ctransferencias_emitidas, 4) over ventana_6 as ctransferencias_emitidas_lag_4, lag(ctransferencias_emitidas, 5) over ventana_6 as ctransferencias_emitidas_lag_5, lag(ctransferencias_emitidas, 6) over ventana_6 as ctransferencias_emitidas_lag_6, avg(ctransferencias_emitidas) over ventana_6 as ctransferencias_emitidas_media_6
, regr_slope(mpayroll, cliente_antiguedad) over ventana_6 as mpayroll_slope_6, lag(mpayroll, 1) over ventana_6 as mpayroll_lag_1, lag(mpayroll, 2) over ventana_6 as mpayroll_lag_2, lag(mpayroll, 3) over ventana_6 as mpayroll_lag_3, lag(mpayroll, 4) over ventana_6 as mpayroll_lag_4, lag(mpayroll, 5) over ventana_6 as mpayroll_lag_5, lag(mpayroll, 6) over ventana_6 as mpayroll_lag_6, avg(mpayroll) over ventana_6 as mpayroll_media_6
, regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_6 as cpayroll_trx_slope_6, lag(cpayroll_trx, 1) over ventana_6 as cpayroll_trx_lag_1, lag(cpayroll_trx, 2) over ventana_6 as cpayroll_trx_lag_2, lag(cpayroll_trx, 3) over ventana_6 as cpayroll_trx_lag_3, lag(cpayroll_trx, 4) over ventana_6 as cpayroll_trx_lag_4, lag(cpayroll_trx, 5) over ventana_6 as cpayroll_trx_lag_5, lag(cpayroll_trx, 6) over ventana_6 as cpayroll_trx_lag_6, avg(cpayroll_trx) over ventana_6 as cpayroll_trx_media_6
, regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_6 as mcomisiones_mantenimiento_slope_6, lag(mcomisiones_mantenimiento, 1) over ventana_6 as mcomisiones_mantenimiento_lag_1, lag(mcomisiones_mantenimiento, 2) over ventana_6 as mcomisiones_mantenimiento_lag_2, lag(mcomisiones_mantenimiento, 3) over ventana_6 as mcomisiones_mantenimiento_lag_3, lag(mcomisiones_mantenimiento, 4) over ventana_6 as mcomisiones_mantenimiento_lag_4, lag(mcomisiones_mantenimiento, 5) over ventana_6 as mcomisiones_mantenimiento_lag_5, lag(mcomisiones_mantenimiento, 6) over ventana_6 as mcomisiones_mantenimiento_lag_6, avg(mcomisiones_mantenimiento) over ventana_6 as mcomisiones_mantenimiento_media_6
, regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_6 as mcomisiones_otras_slope_6, lag(mcomisiones_otras, 1) over ventana_6 as mcomisiones_otras_lag_1, lag(mcomisiones_otras, 2) over ventana_6 as mcomisiones_otras_lag_2, lag(mcomisiones_otras, 3) over ventana_6 as mcomisiones_otras_lag_3, lag(mcomisiones_otras, 4) over ventana_6 as mcomisiones_otras_lag_4, lag(mcomisiones_otras, 5) over ventana_6 as mcomisiones_otras_lag_5, lag(mcomisiones_otras, 6) over ventana_6 as mcomisiones_otras_lag_6, avg(mcomisiones_otras) over ventana_6 as mcomisiones_otras_media_6
, regr_slope(mcomisiones, cliente_antiguedad) over ventana_6 as mcomisiones_slope_6, lag(mcomisiones, 1) over ventana_6 as mcomisiones_lag_1, lag(mcomisiones, 2) over ventana_6 as mcomisiones_lag_2, lag(mcomisiones, 3) over ventana_6 as mcomisiones_lag_3, lag(mcomisiones, 4) over ventana_6 as mcomisiones_lag_4, lag(mcomisiones, 5) over ventana_6 as mcomisiones_lag_5, lag(mcomisiones, 6) over ventana_6 as mcomisiones_lag_6, avg(mcomisiones) over ventana_6 as mcomisiones_media_6
, regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_6 as mcaja_ahorro_slope_6, lag(mcaja_ahorro, 1) over ventana_6 as mcaja_ahorro_lag_1, lag(mcaja_ahorro, 2) over ventana_6 as mcaja_ahorro_lag_2, lag(mcaja_ahorro, 3) over ventana_6 as mcaja_ahorro_lag_3, lag(mcaja_ahorro, 4) over ventana_6 as mcaja_ahorro_lag_4, lag(mcaja_ahorro, 5) over ventana_6 as mcaja_ahorro_lag_5, lag(mcaja_ahorro, 6) over ventana_6 as mcaja_ahorro_lag_6, avg(mcaja_ahorro) over ventana_6 as mcaja_ahorro_media_6
, regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_6 as mcuentas_saldo_slope_6, lag(mcuentas_saldo, 1) over ventana_6 as mcuentas_saldo_lag_1, lag(mcuentas_saldo, 2) over ventana_6 as mcuentas_saldo_lag_2, lag(mcuentas_saldo, 3) over ventana_6 as mcuentas_saldo_lag_3, lag(mcuentas_saldo, 4) over ventana_6 as mcuentas_saldo_lag_4, lag(mcuentas_saldo, 5) over ventana_6 as mcuentas_saldo_lag_5, lag(mcuentas_saldo, 6) over ventana_6 as mcuentas_saldo_lag_6, avg(mcuentas_saldo) over ventana_6 as mcuentas_saldo_media_6
, regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_6 as mtarjeta_visa_consumo_slope_6, lag(mtarjeta_visa_consumo, 1) over ventana_6 as mtarjeta_visa_consumo_lag_1, lag(mtarjeta_visa_consumo, 2) over ventana_6 as mtarjeta_visa_consumo_lag_2, lag(mtarjeta_visa_consumo, 3) over ventana_6 as mtarjeta_visa_consumo_lag_3, lag(mtarjeta_visa_consumo, 4) over ventana_6 as mtarjeta_visa_consumo_lag_4, lag(mtarjeta_visa_consumo, 5) over ventana_6 as mtarjeta_visa_consumo_lag_5, lag(mtarjeta_visa_consumo, 6) over ventana_6 as mtarjeta_visa_consumo_lag_6, avg(mtarjeta_visa_consumo) over ventana_6 as mtarjeta_visa_consumo_media_6
, regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_6 as mprestamos_personales_slope_6, lag(mprestamos_personales, 1) over ventana_6 as mprestamos_personales_lag_1, lag(mprestamos_personales, 2) over ventana_6 as mprestamos_personales_lag_2, lag(mprestamos_personales, 3) over ventana_6 as mprestamos_personales_lag_3, lag(mprestamos_personales, 4) over ventana_6 as mprestamos_personales_lag_4, lag(mprestamos_personales, 5) over ventana_6 as mprestamos_personales_lag_5, lag(mprestamos_personales, 6) over ventana_6 as mprestamos_personales_lag_6, avg(mprestamos_personales) over ventana_6 as mprestamos_personales_media_6
FROM competencia_02
WINDOW ventana_6 AS (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 5 PRECEDING AND CURRENT ROW)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4min 41s, sys: 41.5 s, total: 5min 23s
Wall time: 3min 39s


,Success


## Escribo el archivo

In [ ]:
%%time
%%sql
copy competencia_02 to '/content/competencia_02_FEH.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 17min 39s, sys: 15.5 s, total: 17min 54s
Wall time: 12min


,Success
